In [8]:
import pandas as pd
import numpy as np
import json

In [13]:
df = pd.read_csv('../data/ejercicios_integracion/ventas_tienda_simulada.csv')

In [15]:
print(df.shape)
print(df.head())

(20, 5)
        fecha    producto  cantidad  precio_unitario  total
0  2023-01-01  Producto_A       2.0             10.5   21.0
1  2023-01-02  Producto_B       NaN             20.0    NaN
2  2023-01-03  Producto_C       5.0              NaN    NaN
3  2023-01-04  Producto_D       7.0             15.0  105.0
4  2023-01-05  Producto_A       1.0             12.0   12.0


In [16]:
qsna=df.shape[0]-df.isnull().sum(axis=0)
qna=df.isnull().sum(axis=0)
ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
na=pd.DataFrame(data=aux)
na.sort_values(by='Na en %',ascending=False)

,datos sin NAs en q,Na en q,Na en %
total,12,8,40.0
cantidad,16,4,20.0
precio_unitario,16,4,20.0
fecha,20,0,0.0
producto,20,0,0.0


In [18]:
df.loc[df.isnull()['total']]


,fecha,producto,cantidad,precio_unitario,total
1,2023-01-02,Producto_B,NaN,20.0,NaN
2,2023-01-03,Producto_C,5.0,NaN,NaN
6,2023-01-07,Producto_C,NaN,NaN,NaN
8,2023-01-09,Producto_A,2.0,10.0,NaN
9,2023-01-10,Producto_B,NaN,NaN,NaN
13,2023-01-14,Producto_B,NaN,25.0,NaN
16,2023-01-17,Producto_A,8.0,10.0,NaN
18,2023-01-19,Producto_C,7.0,NaN,NaN


In [33]:
df.loc[df.producto=="Producto_A",:]

,fecha,producto,cantidad,precio_unitario,total
0,2023-01-01,Producto_A,10.5,10.5,21.0
4,2023-01-05,Producto_A,12.0,12.0,12.0
8,2023-01-09,Producto_A,10.0,10.0,NaN
12,2023-01-13,Producto_A,20.0,20.0,100.0
16,2023-01-17,Producto_A,10.0,10.0,NaN


In [36]:
df.loc[df.producto=="Producto_A",'total'] = df.loc[df.producto=="Producto_A",'cantidad'] * df.loc[df.producto=="Producto_A",'precio_unitario']
df.loc[df.producto=="Producto_A",:]

,fecha,producto,cantidad,precio_unitario,total
0,2023-01-01,Producto_A,10.5,10.5,110.25
4,2023-01-05,Producto_A,12.0,12.0,144.00
8,2023-01-09,Producto_A,10.0,10.0,100.00
12,2023-01-13,Producto_A,20.0,20.0,400.00
16,2023-01-17,Producto_A,10.0,10.0,100.00


In [37]:
df.loc[df.producto=="Producto_B",:]

,fecha,producto,cantidad,precio_unitario,total
1,2023-01-02,Producto_B,NaN,20.0,NaN
5,2023-01-06,Producto_B,4.0,25.0,100.0
9,2023-01-10,Producto_B,NaN,NaN,NaN
13,2023-01-14,Producto_B,NaN,25.0,NaN
17,2023-01-18,Producto_B,4.0,11.0,44.0


In [38]:
df.loc[df.producto=="Producto_B",:].dropna(inplace=True)
df.loc[df.producto=="Producto_B",:]

C:\Users\guill\AppData\Local\Temp\ipykernel_15868\3869852627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.producto=="Producto_B",:].dropna(inplace=True)


,fecha,producto,cantidad,precio_unitario,total
1,2023-01-02,Producto_B,NaN,20.0,NaN
5,2023-01-06,Producto_B,4.0,25.0,100.0
9,2023-01-10,Producto_B,NaN,NaN,NaN
13,2023-01-14,Producto_B,NaN,25.0,NaN
17,2023-01-18,Producto_B,4.0,11.0,44.0


In [40]:
df.index.isin(df.dropna().index)

array([ True, False, False,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True])

In [41]:
df.head()

,fecha,producto,cantidad,precio_unitario,total
0,2023-01-01,Producto_A,10.5,10.5,110.25
1,2023-01-02,Producto_B,NaN,20.0,NaN
2,2023-01-03,Producto_C,5.0,NaN,NaN
3,2023-01-04,Producto_D,7.0,15.0,105.00
4,2023-01-05,Producto_A,12.0,12.0,144.00


In [48]:
(df['producto'] == 'Producto_B') & df.index.isin(df.dropna().index)

0     False
1     False
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17     True
18    False
19    False
Name: producto, dtype: bool

In [44]:
~(df['producto'] == 'Producto_B') & df.index.isin(df.dropna().index)


0      True
1     False
2     False
3      True
4      True
5     False
6     False
7      True
8      True
9     False
10     True
11     True
12     True
13    False
14     True
15     True
16     True
17    False
18    False
19     True
Name: producto, dtype: bool